### TextBlob
* use lemma
* use pos tagger
* sentiment analysis
---
* clean slangs
* 

In [1]:
from textblob import TextBlob

In [2]:
#imports
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import sent_tokenize
from nltk.stem import LancasterStemmer, PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.over_sampling import SMOTE


import re

import matplotlib.pyplot as plt
import seaborn as sns

# import plotly
# import plotly.tools as tls   
# import plotly.graph_objs as go

In [3]:
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/Aniket/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [6]:
import sys
sys.path.append("/Users/Aniket/MyStuff/Study/GreyAtom/Hackathon#3/Hack3_gen_functions")


In [7]:
import Hackathon3GeneralFunctions as genf

In [8]:
data = pd.read_csv('data/train.csv')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7274 entries, 0 to 7273
Data columns (total 3 columns):
tweet_id     7274 non-null int64
tweet        7273 non-null object
sentiment    7274 non-null int64
dtypes: int64(2), object(1)
memory usage: 170.6+ KB


In [10]:
data.shape

(7274, 3)

In [11]:
#drop null
data.drop(1274,axis=0,inplace=True)

data.isnull().sum()

tweet_id     0
tweet        0
sentiment    0
dtype: int64

In [12]:
data.shape

(7273, 3)

In [13]:
data['clean_tweet'] = data.tweet.map(lambda x: genf.clean_text(x,lower=False))

#### Textblob test

In [14]:
for i in range(5):
    wiki = TextBlob(data.loc[i,'clean_tweet'])
    print(f'POS tags for sent {i+1}')
    print(wiki.tags)
    print('---')
    print(f'Nounphrases for sent')
    print(wiki.noun_phrases)
    print('===')
    print()



POS tags for sent 1
[('sxswnui', 'NN'), ('sxsw', 'NN'), ('apple', 'NN'), ('defining', 'VBG'), ('language', 'NN'), ('of', 'IN'), ('touch', 'JJ'), ('with', 'IN'), ('different', 'JJ'), ('dialects', 'NNS'), ('becoming', 'VBG'), ('smaller', 'JJR')]
---
Nounphrases for sent
['sxswnui sxsw apple', 'different dialects']
===

POS tags for sent 2
[('Learning', 'VBG'), ('ab', 'JJ'), ('Google', 'NNP'), ('doodles', 'VBZ'), ('All', 'DT'), ('doodles', 'NNS'), ('should', 'MD'), ('be', 'VB'), ('light', 'JJ'), ('funny', 'JJ'), ('innovative', 'JJ'), ('with', 'IN'), ('exceptions', 'NNS'), ('for', 'IN'), ('significant', 'JJ'), ('occasions', 'NNS'), ('GoogleDoodle', 'NNP'), ('sxsw', 'NN')]
---
Nounphrases for sent
['learning', 'google', 'light funny innovative', 'significant occasions', 'googledoodle']
===

POS tags for sent 3
[('one', 'CD'), ('of', 'IN'), ('the', 'DT'), ('most', 'RBS'), ('in', 'IN'), ('your', 'PRP$'), ('face', 'NN'), ('ex', 'NN'), ('of', 'IN'), ('stealing', 'VBG'), ('the', 'DT'), ('show', 

In [15]:
for i in range(5):
    wiki = TextBlob(data.loc[i,'clean_tweet'])
    print(f"Sent {i+1}:",wiki)
    print(f'Sentiment for sent {i+1}: {wiki.sentiment}')
    print(f"Actual sentiment: {data.loc[i,'sentiment']}")
    print('===')
    print()

Sent 1:  sxswnui  sxsw  apple defining language of touch with different dialects becoming smaller
Sentiment for sent 1: Sentiment(polarity=0.15, subjectivity=0.65)
Actual sentiment: 1
===

Sent 2: Learning ab Google doodles  All doodles should be light  funny   innovative  with exceptions for significant occasions   GoogleDoodle  sxsw
Sentiment for sent 2: Sentiment(polarity=0.38125, subjectivity=0.89375)
Actual sentiment: 1
===

Sent 3: one of the most in your face ex  of stealing the show in yrs RT    At  SXSW  Apple schools the mkt experts    
Sentiment for sent 3: Sentiment(polarity=0.5, subjectivity=0.5)
Actual sentiment: 2
===

Sent 4: This iPhone  SXSW app would b pretty awesome if it didnt crash every 10mins during extended browsing   Fuckit  Illmakeitwork
Sentiment for sent 4: Sentiment(polarity=0.625, subjectivity=1.0)
Actual sentiment: 0
===

Sent 5: Line outside the Apple store in Austin waiting for the new iPad  SXSW   
Sentiment for sent 5: Sentiment(polarity=0.0681818181

In [16]:
w = TextBlob("This is a sampleee! I looooveee iPhone ipad iOS")
w.correct()

TextBlob("His is a sample! I looooveee shone pad in")

In [17]:
# TextBlob("The new iOS is good. Good job Apple apples").tags

In [18]:
_slang_loopup(w)

NameError: name '_slang_loopup' is not defined

In [36]:
from gingerit.gingerit import GingerIt
parser = GingerIt()
line= "I loooveee Apple's new iPad ipad 2 with AT&T/GSM&D deal #ipad2 having"
tweet=parser.parse(line)
tweet

{'text': "I loooveee Apple's new iPad ipad 2 with AT&T/GSM&D deal #ipad2 having",
 'result': "I love Apple's new iPad iPad 2 with AT&T/GSM&D deal #ipad2 having",
 'corrections': [{'start': 28,
   'text': 'ipad',
   'correct': 'iPad',
   'definition': None},
  {'start': 2,
   'text': 'loooveee',
   'correct': 'love',
   'definition': 'have a great affection or liking for'}]}

In [37]:
tweet['result']

"I love Apple's new iPad iPad 2 with AT&T/GSM&D deal #ipad2 having"